In [1]:
import os
import re
import pdfplumber as pp
import pandas as pd

In [2]:
def detect_pattern(lines):
    for i, line in enumerate(lines):
        if re.match(r"\d{2}-[A-Za-z]{3}-\d{4}", line.strip()):
            if i + 1 < len(lines):
                next_line = lines[i + 1]
                if re.findall(r"[\d,]+\.\d{2}", next_line) and re.search(r"(Dr|Cr)$", next_line):
                    return "PatternA"
    return "Unknown"


In [3]:
def parse_pattern_a(lines):
    data = []
    i = 0
    while i < len(lines) - 1:
        line1 = lines[i].strip()
        line2 = lines[i + 1].strip()

        if re.match(r"\d{2}-[A-Za-z]{3}-\d{4}", line1):
            date = re.match(r"(\d{2}-[A-Za-z]{3}-\d{4})", line1).group(1)
            description = line1[len(date):].strip()

            amounts = re.findall(r"[\d,]+\.\d{2}", line2)
            drcr_match = re.search(r"(Dr|Cr)$", line2)
            drcr = drcr_match.group(1) if drcr_match else ""

            if len(amounts) == 2:
                amount = amounts[0]
                balance = amounts[1]
            else:
                amount = balance = ""

            data.append({
                "Date": date,
                "Description": description,
                "Amount": amount.replace(",", ""),
                "Balance": balance.replace(",", ""),
                "Type": drcr
            })

            i += 2
        else:
            i += 1
    return data

In [4]:
def extract_text_lines(pdf_path):
    with pp.open(pdf_path) as pdf:
        text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())
    return text.split("\n")

In [ ]:
def main():
    pdf_path = "test.pdf"

    lines = extract_text_lines(pdf_path)
    pattern = detect_pattern(lines)

    if pattern == "PatternA":
        data = parse_pattern_a(lines)
        df = pd.DataFrame(data)
        output_path = os.path.splitext(pdf_path)[0] + "_parsed.xlsx"
        df.to_excel(output_path, index=False)
        print(f"Extracted data saved to: {output_path}")
    else:
        print("Unknown statement format. Please provide a sample to train the parser.")


In [6]:
if __name__ == "__main__":
    main()

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


✅ Extracted data saved to: test2_parsed.xlsx
